In [1]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import time
import pickle

Adding Configuration

In [ ]:

IMG_SIZE = (64, 64)  # Resize images to this size
NUM_CLASSES = 2      # Number of gestures/poses to classify
SAMPLES_PER_CLASS = 50  # How many images to collect per class

CLASS_NAMES = ["Angry", "Happy"]  # Names of the classes

print("="*60)
print("LINEAR SVM WEBCAM CLASSIFIER")
print("="*60)
print(f"\nWe'll train a model to classify: {CLASS_NAMES}")
print(f"Collecting {SAMPLES_PER_CLASS} samples per class\n")

LINEAR SVM WEBCAM CLASSIFIER

We'll train a model to classify: ['Angry', 'Happy']



Data Collection

In [ ]:

def collect_training_data():
    """Collect training images from webcam"""
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("ERROR: Could not open webcam!")
        return None, None
    
    X_data = []
    y_data = []
    
    for class_idx, class_name in enumerate(CLASS_NAMES):
        print(f"\n{'='*60}")
        print(f"CLASS {class_idx + 1}: {class_name}")
        print(f"{'='*60}")
        print(f"Press SPACE to start collecting {SAMPLES_PER_CLASS} samples")
        print("Press 'q' to quit\n")
        
        collected = 0
        collecting = False
        
        while collected < SAMPLES_PER_CLASS:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame")
                break
            
            # Flip frame for mirror effect
            frame = cv2.flip(frame, 1)
            
            # Display info on frame
            cv2.putText(frame, f"Class: {class_name}", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Collected: {collected}/{SAMPLES_PER_CLASS}", (10, 70),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            if collecting:
                cv2.putText(frame, "COLLECTING...", (10, 110),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                cv2.putText(frame, "Press SPACE to start", (10, 110),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            
            cv2.imshow('Data Collection', frame)
            
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord(' '):
                collecting = True
            elif key == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                return None, None
            
            if collecting:
                # Convert to grayscale and resize
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                resized = cv2.resize(gray, IMG_SIZE)
                
                # Flatten image to 1D array
                flattened = resized.flatten()
                
                X_data.append(flattened)
                y_data.append(class_idx)
                collected += 1
                
                time.sleep(0.1)  # Small delay between captures
    
    cap.release()
    cv2.destroyAllWindows()
    
    return np.array(X_data), np.array(y_data)

Here we are implementing a Support Vector Machine (SVM) model for classification, trained on data (images from a webcam) and then used for real-time prediction via the webcam feed.

In [ ]:
# ============================================
# TRAIN SVM MODEL
# ============================================
def train_svm_model(X, y):
    """Train Linear SVM classifier"""
    print("\n" + "="*60)
    print("TRAINING LINEAR SVM MODEL")
    print("="*60)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train Linear SVM
    print("\nTraining Linear SVM (this may take a moment)...")
    model = SVC(kernel='linear', C=1.0, probability=True, random_state=42)
    model.fit(X_train_scaled, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"\n✓ Training complete!")
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=CLASS_NAMES))
    
    # Get support vectors info
    print(f"\nNumber of support vectors: {model.n_support_}")
    print(f"Support vectors per class: {dict(zip(CLASS_NAMES, model.n_support_))}")
    
    return model, scaler


def real_time_prediction(model, scaler):
    """Run real-time classification on webcam feed"""
    print("\n" + "="*60)
    print("REAL-TIME CLASSIFICATION")
    print("="*60)
    print("Press 'q' to quit\n")
    
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("ERROR: Could not open webcam!")
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Flip frame for mirror effect
        frame = cv2.flip(frame, 1)
        
        # Preprocess frame
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, IMG_SIZE)
        flattened = resized.flatten().reshape(1, -1)
        
        # Scale and predict
        scaled = scaler.transform(flattened)
        prediction = model.predict(scaled)[0]
        probabilities = model.predict_proba(scaled)[0]
        confidence = probabilities[prediction] * 100
        
        # Get decision function (distance from hyperplane)
        decision = model.decision_function(scaled)[0]
        
        # Display results
        predicted_class = CLASS_NAMES[prediction]
        color = (0, 255, 0) if confidence > 70 else (0, 165, 255)
        
        cv2.putText(frame, f"Prediction: {predicted_class}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.putText(frame, f"Confidence: {confidence:.1f}%", (10, 70),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.putText(frame, f"Distance from hyperplane: {decision:.2f}", (10, 110),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Show probability bars
        y_pos = 150
        for i, (class_name, prob) in enumerate(zip(CLASS_NAMES, probabilities)):
            bar_length = int(prob * 300)
            cv2.rectangle(frame, (10, y_pos), (10 + bar_length, y_pos + 30), 
                         (0, 255, 0), -1)
            cv2.putText(frame, f"{class_name}: {prob*100:.1f}%", (10, y_pos + 20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            y_pos += 50
        
        cv2.imshow('SVM Real-Time Classification', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()


def main():
    print("\nStarting in 3 seconds...")
    time.sleep(3)
    
    # Step 1: Collect data
    print("\nSTEP 1: DATA COLLECTION")
    X_data, y_data = collect_training_data()
    
    if X_data is None:
        print("Data collection cancelled.")
        return
    
    print(f"\n✓ Collected {len(X_data)} total samples")
    print(f"Feature vector size: {X_data.shape[1]}")
    
    # Step 2: Train model
    model, scaler = train_svm_model(X_data, y_data)
    
    # Optional: Save model
    save_model = input("\nSave model? (y/n): ").lower()
    if save_model == 'y':
        with open('svm_webcam_model.pkl', 'wb') as f:
            pickle.dump((model, scaler, CLASS_NAMES), f)
        print("✓ Model saved to 'svm_webcam_model.pkl'")
    
    # Step 3: Real-time prediction
    start_prediction = input("\nStart real-time prediction? (y/n): ").lower()
    if start_prediction == 'y':
        real_time_prediction(model, scaler)
    
    print("\n" + "="*60)
    print("DONE!")
    print("="*60)


Testing the SVM model by running the program

In [ ]:

if __name__ == "__main__":
    main()


Starting in 3 seconds...

STEP 1: DATA COLLECTION

CLASS 1: Angry
Press SPACE to start collecting 50 samples
Press 'q' to quit


CLASS 2: Happy
Press SPACE to start collecting 50 samples
Press 'q' to quit


✓ Collected 100 total samples
Feature vector size: 4096

TRAINING LINEAR SVM MODEL

Training Linear SVM (this may take a moment)...

✓ Training complete!
Test Accuracy: 1.0000

Classification Report:
              precision    recall  f1-score   support

       Angry       1.00      1.00      1.00        10
       Happy       1.00      1.00      1.00        10

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20


Number of support vectors: [6 5]
Support vectors per class: {'Angry': np.int32(6), 'Happy': np.int32(5)}
✓ Model saved to 'svm_webcam_model.pkl'

REAL-TIME CLASSIFICATION
Press 'q' to quit


DONE!
